# Homework 6 (Power calculations)
## J. Eduardo Rodriguez Almaraz

### Set up the program with all the required libraries

In [1]:
import random
import numpy as np
import pandas as pd
import statistics
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

### Set all the necessary parameters using the values from the paper

* The prevalence of homozygosity of the allele is 0.09 (based on a minor allele frequency of 0.3: 0.3*0.3=0.09). 


In [2]:
prev_hmz = 0.09

We will create our sample size here.\
For this example we will use a sample size of 25 but the main program will be customizable

In [3]:
sample_size = 250

* The mean of CRP for people who are not hozygotes is 2.01(SD=2.09)

We will create a random distribution using these two parameters

In [4]:
random.seed(44)
crp_mean = 2.01
crp_sd = 2.09

#This function takes the arguments: mean, SD, and n to create the distribution
crp_dist = np.random.normal(crp_mean, crp_sd, sample_size)

* The type I error rate is = 0.05

In [5]:
alpha_lvl = 0.05

With this parameters now we can create a simulation.

The first step is to create a list of random uniformly distributed numbers where the lenght of the list is the same as the sample size.



In [6]:
#Creates an empty list
randomlist = []

# Appends the list created above with a uniformly distributed rnadom number between 0 and 1 rounding to the 4th decimal place
for i in range(0,sample_size):
    n = round(random.uniform(0.0001,.9999),4)
    randomlist.append(n)

*If you want to check the list uncomment the line below*

In [7]:
#print(randomlist)

Now lets create a list of values based on the prevalence of homozygosity of the allele using a similar process as above

In [8]:
all_hz = []
for n in randomlist:
    if n<prev_hmz:
        all_hz.append(1)
    else:
        all_hz.append(0)
        
        

*If you want to check the list uncomment the line below. You also can check that both lists are the same lenght*

In [9]:
#print(all_hz)        
#len(all_hz)
#len(randomlist)

Just to checek that the prevalence is close to the original 0.09. 

However, since we created random numbers it will not be exactly 0.09

In [10]:
print("The prevalence of Homozygocity for this allele is", round(all_hz.count(1)/sample_size,2))

The prevalence of Homozygocity for this allele is 0.12


Now lets put everything together on a more legible dataframe

In [11]:
main_df = pd.DataFrame({'Allele_hz':all_hz, 'CRP_distrib':crp_dist})
main_df.head()

,Allele_hz,CRP_distrib
0,0,0.759967
1,0,2.262789
2,0,1.613854
3,0,6.918580
4,0,1.695753


In [12]:
print('Mean of CRP distribution among non-homozygotes',statistics.mean(crp_dist))
print('Standard deviation of CRP distribution among non-homozygotes',statistics.stdev(crp_dist))
print('Median of CRP distribution among non-homozygotes',statistics.median(crp_dist))


Mean of CRP distribution among non-homozygotes 2.1716589575147154
Standard deviation of CRP distribution among non-homozygotes 1.9975344275910492
Median of CRP distribution among non-homozygotes 2.2055320354662875


Now lets work with the outcome using the information from the instructions:

* The 10-year cumulative incidence of MI for people who are not homozygotes is 2.70%.

Under this information we can assume that the probability of MI for Allele heterozyotes (`all_hz=0`) is 0.027

In [13]:
random.seed(43)
random_y =[]

for i in all_hz:
    if i==0:
        random_y.append(round(random.uniform(0.00001,.99999),6))
    else:
        random_y.append(round(random.uniform(0.00001,.9999),6))
        

*Note: Eventhough is the same formula for both conditions (i=1 and i=0) the loop is generating 2 independent random numbers in each iteration

Now lets add the outcome MI based on these probability. 
The notation below states: 
* Add the `random_y` variable to the data frame
* For Allele heterogocytes (IV=0) if `random_y` is equal or less than the prevalence create a variable with the outcome (MI) = 1
Since we know that the  probability of having MY being heterogyzgous is 0.027 the the odds ratio:

$OR=$ $\frac{0.027}{1-0.027}$ $=0.02774923$

Thus 

$P(Y|Allele=1) = 0.02774923^*(OR_{homozygous})$

$=0.02774923 ^* 1.25 =$ **0.03468654** 



In [14]:
main_df['random_y']= random_y
main_df['MI_y'] = np.where((main_df['Allele_hz']==0) & (main_df['random_y']<0.027) | (main_df['Allele_hz']==1) & (main_df['random_y']<0.03468654),1,0)
main_df.head(20)

,Allele_hz,CRP_distrib,random_y,MI_y
0,0,0.759967,0.038561,0
1,0,2.262789,0.696220,0
2,0,1.613854,0.143940,0
3,0,6.918580,0.462533,0
4,0,1.695753,0.671643,0
5,1,4.232874,0.792874,0
6,0,6.893470,0.453190,0
7,1,2.913878,0.498227,0
8,0,0.539981,0.019167,1
9,0,2.268647,0.432390,0


In [15]:
print('Incidence of outcome among Allele Hz=0', len(main_df.loc[(main_df['Allele_hz']==0) & (main_df['MI_y']==1), ['Allele_hz','MI_y']])/sample_size)
print('Incidence of outcome among Allele Hz=1', len(main_df.loc[(main_df['Allele_hz']==1) & (main_df['MI_y']==1), ['Allele_hz','MI_y']])/sample_size)\


table = pd.crosstab(main_df['Allele_hz'], main_df['MI_y'])
print('')

print(table)

Incidence of outcome among Allele Hz=0 0.02
Incidence of outcome among Allele Hz=1 0.004

MI_y         0  1
Allele_hz        
0          215  5
1           29  1


## Creating Models to predict

### We will use the two-stage model process

### First Stage-model

We need two coefficients:

* $\hat{\beta}_{Y|Z}$

And
* $\hat{\beta}_{X|Z}$

Where: 

**Y= MI**

**Z = Allele homogygocity**

**X = CRP**

Lets begin by regressing the IV (`allele_hz`) on the exposure (`crp`):

Creates all of our variables to be used in the model

In [16]:
y = np.array(main_df['MI_y'])
x = np.array(main_df['CRP_distrib'])
z = np.array(main_df['Allele_hz'])

## First-stage model

* Initialize the model
* Fit the model. The first argument is the outcome and the second is the predictor. In this case we are regressing Allele on CRP
* Predict the values of CRP (X). A list has been created The last line checks that is the same size as the dataset

In [17]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
#from sklearn.linear_model import LogisticRegression
#from sklearn import metrics

model_x_z = smf.ols(formula = "CRP_distrib~Allele_hz",
                data= main_df).fit()

#z_pred = model_x_z.predict(x)
z_pred = model_x_z.predict()
main_df['z_pred'] = z_pred
model_x_z.params
len(z_pred)


250

Now we will fit a model to estimate the outcome using first our IV to check that the OR is close to the original one of 1.25

In [18]:
import statsmodels.api as sm
from statsmodels.formula.api import glm

model_z_y = glm(formula = 'MI_y~Allele_hz',
                data=main_df,
               family = sm.families.Binomial()).fit()

print(model_z_y.summary())
print('')
print('The OR that results from the model above is:',np.exp(model_z_y.params[1]))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   MI_y   No. Observations:                  250
Model:                            GLM   Df Residuals:                      248
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -28.248
Date:                Thu, 27 May 2021   Deviance:                       56.496
Time:                        00:34:00   Pearson chi2:                     250.
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.7612      0.452     -8.314      0.0

**This looks good!!!                          ^**

## Second-Stage model

We just fit a logistic regression of the predicted exposures that we stored in `z_pred` on the actual outcome

In [19]:
from statsmodels.formula.api import glm
model_zpred_y = glm(formula = 'MI_y~z_pred',
                data=main_df,
               family = sm.families.Binomial()).fit()

zpred_y = model_zpred_y.predict()
print(model_zpred_y.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   MI_y   No. Observations:                  250
Model:                            GLM   Df Residuals:                      248
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -28.248
Date:                Thu, 27 May 2021   Deviance:                       56.496
Time:                        00:34:00   Pearson chi2:                     250.
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5867      8.797     -0.067      0.9

In [20]:
h1 = (model_zpred_y.pvalues[1])<0.05
h1
#model_zpred_y.conf_int(cols=[0,0])

False

In [21]:
model_zpred_y.params[1]

-1.4400075709687625

### Simulations

For simulations we neeed to compile all the code above and set 2 parameters: 

* The number of times we want to run out code

* The sample sizes that we want to use for each simulation

The code below will show the same that we did above but 'enclosed' on a function.

In [22]:
     
def _power_calculation_ (nobs):
    random.seed(44)
    prev_hmz = 0.09
    sample_size = nobs
    crp_mean = 2.01
    crp_sd = 2.09
    crp_dist = np.random.normal(crp_mean, crp_sd, sample_size)
    alpha_lvl = 0.05
    randomlist = []
    for i in range(0,sample_size):
        n = round(random.uniform(0.0001,.9999),4)
        randomlist.append(n)

    all_hz = []
    for n in randomlist:
        if n<prev_hmz:
            all_hz.append(1)
        else:
            all_hz.append(0)

    main_df = pd.DataFrame({'Allele_hz':all_hz, 'CRP_distrib':crp_dist})
    random_y =[]

    for i in all_hz:
        if i==0:
            random_y.append(round(random.uniform(0.00001,.99999),6))
        else:
            random_y.append(round(random.uniform(0.00001,.9999),6))

    main_df['random_y']= random_y
    main_df['MI_y'] = np.where((main_df['Allele_hz']==0) & (main_df['random_y']<0.027) | (main_df['Allele_hz']==1) & (main_df['random_y']<0.03468654),1,0)

    model_x_z = smf.ols(formula = "CRP_distrib~Allele_hz",
                data= main_df).fit()


    z_pred = model_x_z.predict()
    main_df['z_pred'] = z_pred
    model_z_y = glm(formula = 'MI_y~Allele_hz',
                data=main_df,
               family = sm.families.Binomial()).fit()

    model_zpred_y = glm(formula = 'MI_y~z_pred',
                data=main_df,
               family = sm.families.Binomial()).fit()

    zpred_y = model_zpred_y.predict()
    coefficients = model_zpred_y.params[1]
    h1 = (model_zpred_y.pvalues[1])<0.05
    conf_int = model_zpred_y.conf_int(alpha=0.05)
    
    
    return dict({'coefficients':coefficients,'H1 rejected':h1, 'Confidence interval':conf_int})

#Defines a function that takes two arguments: Number of simulations and number of observations


def pwr(sims, obs):
    #Creates empy lists to be appended to then itterate trhough them
    alt_hyp = []
    beta = []
    conf_int = []
    for i in range(sims):
        pwr_test = _power_calculation_(obs)
        alt_hyp.append(pwr_test['H1 rejected'])
        conf_int.append(pwr_test['Confidence interval'])
        beta.append(pwr_test['coefficients'])


    print('With',sims,'simulations and',obs, 'observations:')
    print('')
    
    #The list alt_hyp is a logical list that takes values of 0 and 1 where 1 is true, thus if
    #more than half of the simulations reject the null we call the pool 'True'
    if np.average(alt_hyp)>0.5:
        print('Null rejected')
    else:
        print('Fail to reject null')

        #Prints the average of the coefficients...Not sure how to pool the confidence intervals although, they are appended before.
    print('Average OR:',np.mean(beta))
  

### Now we can try all put together
The function `pwr` takes 2 arguments: 1) The number of simulations and 2) the number of observations...this migth take a while

In [23]:
pwr(1000,2000)

With 1000 simulations and 2000 observations:

Null rejected
Average OR: 1.3742869747959783
